In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import shutil
from operator import itemgetter
from pathlib import Path

import dask.dataframe as ddf
import dask.delayed as delayed
import fiona
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycountry as pyctry
import rioxarray
import xarray as xr
from dask_gateway import Gateway
from py7zr import unpack_7zarchive
from tqdm.auto import tqdm
import PyPDF2 as pypdf

from sliiders import country_level_ypk as ypk_fn
from sliiders import settings as sset
from sliiders import spatial

import requests
from zipfile import ZipFile
from io import BytesIO

from bs4 import BeautifulSoup as BSoup

# dask gateway setup
gateway = Gateway()
image_name = sset.DASK_IMAGE

import re

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


## Downloading

In [56]:
cia_download_url = "https://www.cia.gov/the-world-factbook/about/archives/download"
cia_files = [f"factbook-{x}.zip" for x in range(2000, 2021)]

In [ ]:
os.makedirs(sset.DIR_CIA_RAW, exist_ok=True)

for i in tqdm(cia_files):
    cia_req = requests.get("/".join([cia_download_url, i]))
    cia_zip = ZipFile(BytesIO(cia_req.content))
    cia_zip.extractall(str(sset.DIR_CIA_RAW))

### WFB 2002

<html>
<head>
<title>CIA -- The World Factbook -- Aruba</title>
</head>
<body bgcolor="cccccc" link="ffffff" vlink="cccccc">
<table bgcolor="666666" border="0" cellpadding="2" cellspacing="0" width="100%">
<tr>
<td valign="middle" width="20%"><img alt="CIA Seal" border="0" src="../ciaseal.gif"/><img alt="Factbook Logo" border="0" src="../fblogo.gif"/></td>
<td align="center" valign="middle" width="60%"><font color="ffffff" face="arial" size="5">Aruba</font><br/><font color="ffffff" face="arial" size="2"><i>(part of the Kingdom of the Netherlands)</i></font></td>
<td align="right" valign="middle" width="20%"><a href="../flags/aa-lgflag.jpg"><img alt="Flag of Aruba" border="0" height="54" src="../flags/aa-flag.jpg"/></a></td>
</tr>
<tr>
<td align="center" colspan="3" width="100%"><font face="arial" size="2">  
<a href="#Intro">Introduction</a>  
<a href="#Geo">Geography</a>  
<a href="#People">People</a>  
<a href="#Govt">Government</a>  
<a href="#Econ">Economy</a>  
<a href="#Comm">Com